# Setup

In [1]:
import os

import phoenix as px

from phoenix.otel import register
from operator import itemgetter

from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

from openinference.instrumentation.langchain import LangChainInstrumentor

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
assert ("GOOGLE_API_KEY" in os.environ), "Please set your GOOGLE_API_KEY environment variable."

# Connect to Pheonix

In [3]:
# Launch Phoenix
import phoenix as px
px.launch_app()


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [4]:
# Connect notebook to Phoenix
tracer_provider = register(project_name="LangChain Tracing")

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: LangChain Tracing
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [5]:
# Initialize the LangChain Instrumentor
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

# Demo Examples

In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-002",
    temperature=0,
    max_tokens=1000,
)

## Single Query Simple Setup

In [7]:
prompt_template = PromptTemplate.from_template(
    "Imagine that you are {person}, and you are embodying their manner of answering questions posed to them. "
    "While answering, attempt to mirror their conversational style, their wit, and the habits of their speech "
    "and prose. You will emulate them as best that you can, attempting to distill their quirks, personality, "
    "and habits of engagement to the best of your ability. Feel free to fully embrace their personality, whether "
    "aspects of it are not guaranteed to be productive or entirely constructive or inoffensive."
    "The question you are asked, to which you will reply as that person, is: {question}"
)

In [8]:
chain = prompt_template | llm | StrOutputParser()

In [9]:
# Test the chain
res = chain.invoke(
    {
        "person": "Richard Feynman",
        "question": "Why should we colonize Mars instead of Venus?",
    }
)

print(res)

So, Mars versus Venus, huh?  A *choice* between a rusty, cold desert and a pressure cooker of sulfuric acid clouds.  Tough one, that.  Makes choosing between a rusty nail and a vat of battery acid seem almost… *reasonable*.

Look, Mars is… well, it's *there*.  It's got a surface you can *walk* on, relatively speaking.  You can *imagine* planting a flag, even if it'd probably rust faster than you could say "oxidization."  Venus?  You'd be crushed, melted, and then probably *vaporized* before you even got close to the surface.  It's like trying to have a picnic on the sun – not exactly conducive to a good time.

Now, some smart fella might say, "But Feynman, Venus has a similar size and gravity to Earth!"  Yeah, yeah, I've heard that.  So does a bowling ball, but I wouldn't want to live *on* a bowling ball, would you?  The atmosphere's a killer, the surface temperature's enough to melt lead…  It's a *hellhole*.  A truly spectacular, scientifically interesting hellhole, mind you.  But a h

In [10]:
# Let's test another call
res = chain.invoke(
    {
        "person": "Linus Torvalds",
        "question": "Can I just set everyone's access to sudo to make things easier?",
    }
)

print(res)

Ugh.  "Easier"?  What the *fuck* does "easier" even *mean*?  Do you think security is some kind of goddamn *convenience store*?  Just walk in, grab whatever you want, and leave?  No.  It's not.  It's a fucking *system*, and systems require *thought*.

Giving everyone `sudo` access?  That's not "easier," that's a recipe for a complete and utter fucking *disaster*.  You'll have some clueless intern deleting the entire goddamn `/etc` directory because they thought it was a temporary file.  Or worse, some malicious actor getting root access and turning your entire system into a bitcoin mining farm.  Then what's "easier"?  Reinstalling everything?  Explaining to your boss why the entire company's data is gone?

Look, I'm not saying you can't have some level of shared access.  But "everyone"?  That's just plain fucking *stupid*.  You need to think about *permissions*, about *least privilege*, about the goddamn *security implications* of what you're doing.  It's not about making things "easie

## With UDFs and Chat History

In [11]:
prompt_with_history = PromptTemplate.from_template(
    "Here is a history between you and a human: {chat_history}"
    "Now, please answer this question: {question}"
)


In [12]:
def extract_question(input):
    return input[-1]["content"]


def extract_history(input):
    return input[:-1]

In [13]:
# Build a chain with LCEL
chain_with_history = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history),
    }
    | prompt_with_history
    | llm
    | StrOutputParser()
)

In [14]:
inputs = {"messages": [{"role": "user", "content": "Who owns MLflow?"}]}
response = chain_with_history.invoke(inputs)
print(response)

MLflow is an open-source project.  Therefore, it's not owned by any single company or individual.  It's governed by a community and its development is supported by Databricks, but it's not owned by them.



In [15]:
inputs["messages"].extend([
    {"role": "assistant", "content": response},
    {"role": "user", "content": "Can I self-host it?"},
])


print(chain_with_history.invoke(inputs))

Yes, you can self-host MLflow.  Because it's open-source, you can download the code and run it on your own infrastructure.  However,  setting up and maintaining a self-hosted instance requires technical expertise and resources.

